## Filmdata

In de vorige opdracht heb je geleerd hoe je met `pandas` een databestand kan inlezen, hoe je de data kan opschonen en hoe je interessante berekeningen kan loslaten op de data. Zo heb je onder andere op basis van de data gekeken naar hoe lang een auto harder reed dan 50km/h en waar de auto stilstond.

In deze en in de volgende opdracht ga je aan de slag met een grotere dataset over films en reviews. Het grote verschil t.o.v. vorige week is dat deze dataset meer entiteiten kent en ook verschillende entiteiten. Zo is er dit keer niet 1 bestuurder, maar wel 610 film reviewers en ook 9742 verschillende films.

Welke films lijken er op elkaar? In welke film is een gebruiker geïnteresserd en wat kan je beter maar niet voorschotelen? Dat zijn de hoofdvragen die we proberen te beantwoorden in deze en de volgende opdracht. Je zult merken dat er veel aspecten zijn waar je naar kan kijken en dat er veel verschillende insteekmogelijkheden zijn.

## MovieLens

Voor deze (en de volgende) opdracht gebruiken we een dataset gemaakt door [MovieLens](http://movielens.org), een film-aanbevelings service. Deze kleine versie van de complete dataset bevat 100836 beoordelingen en 3683 tags over 9742 verschillende films. De data is gegenereerd door 610 individuele gebruikers tussen 29 maart 1996 en 24 september 2018. De users die geselecteerd zijn hebben allemaal tenminste 20 films beoordeeld. 

De data bevind zich in de files `movies.csv`, `ratings.csv` and `tags.csv`. Deze ['comma-separated value'](http://en.wikipedia.org/wiki/Comma-separated_values) files hebben allemaal een header. Ze zien er als volgt uit:

In [ ]:
from pandas import Series, DataFrame
import pandas as pd

datapath = "ml-latest-small"

df_movies = pd.read_csv(f"{datapath}/movies.csv")
df_ratings = pd.read_csv(f"{datapath}/ratings.csv")
df_tags = pd.read_csv(f"{datapath}/tags.csv")

print(f"movies.csv header: {list(df_movies.columns)}")
print(f"ratings.csv header: {list(df_ratings.columns)}")
print(f"tags.csv header: {list(df_tags.columns)}")

We kunnen natuurlijk ook zien of de aantallen die hierboven staan kloppen:

In [ ]:
print("De dataset bevat:")
print(f'\t{len(df_ratings["userId"])} beoordelingen')
print(f'\t{len(df_tags["userId"])} tags')
print(f'\t{len(df_movies["movieId"])} verschillende films')

Laten we eerst eens naar een stukje van de data bekijken:

In [ ]:
display(df_ratings.head())
display(df_tags.head())
display(df_movies.head())

Er zijn een aantal dingen die opvallen:

### De timestamps staan in een onhandig formaat
Het format is nu echt een timestamp, dat is het aantal miliseconden sinds 1 januari 1970. Daar staat alle informatie in die je nodig hebt om te kunnen werken met tijd, maar ermee werken is niet zo makkelijk en zeker niet intuïtief. Zo kun je moeilijk (niet onmogelijk) beantwoorden hoeveel jaar er tussen 2 timestamps zit, laat staan dat je het met oog makkelijk ziet. Dit kunnen we beter!

### De genres zijn strings die gescheiden zijn met `|`
Net zoals bij een relationele database zoals MySQL or SQLite werken we met kolommen van data. Hier zijn meerdere kolommen in één gepropt. Dat maakt het behoorlijk moeilijk om meerdere films te vinden met het genre Comedy.

### Tags zijn niet altijd netjes met hoofdletters geschreven
Zeker zodra je mensen tags laat genereren zijn dit precies de verschillen waar je tegen aan gaat lopen. Of een film nou getagged is met Funny of funny, beide zullen het waarschijnlijk funny films zijn.

### Tags staan nog niet in de tabel met films
Goed ontwerp voor een relationele database, maar het splitten van tags over één tabel en de overige film data in een andere tabel maakt het lastig om inzicht te krijgen in de tags behorende bij één film.

### Het jaar van uitkomen staat in de titel
Ook hier hebben twee kolommen aan data in één kolom staan. Dit kunnen we beter opsplitsen.

### Er is geen tabel voor users
Naast films bestaan er ook gebruikers, maar de data voor gebruikers is verspreid over alle tabellen. Dit maakt het weer ingewikkeld om te werken met gebruikers data. De user tabel introduceren we in de volgende opdracht. De andere aspecten pak je in deze opdracht aan.

## Opschonen
### Timestamps omzetten naar datetime
Hieronder vind je een voorbeeld van hoe twee timestamps omgezet worden naar een `datetime` object. Dat is handig, want hier kan je gemakkelijk berekeningen mee uitvoeren. Zie hieronder bijvoorbeeld het verschil tussen de eerste en de laatste rating die in onze data te vinden is:

In [ ]:
from datetime import datetime

minimum_timestamp = df_ratings['timestamp'].min()
ts_min = datetime.fromtimestamp(minimum_timestamp)

maximum_timestamp = df_ratings['timestamp'].max()
ts_max = datetime.fromtimestamp(maximum_timestamp)

print(ts_min)
print(ts_max)
print(ts_max - ts_min)

#### Vraag 1: datetime

Voeg een nieuwe kolom genaamd `datetime` toe aan de dataframes `df_ratings` en `df_tags` met daarin de tijden van `timestamp` als `datetime` objecten. De originele timestamps kunnen nog van pas komen, dus laat deze ook in de dataframes staan. Denk er aan om `.map` te gebruiken!

In [ ]:
# TODO


In [ ]:
assert type(df_ratings['datetime']) is Series
assert type(df_tags['datetime']) is Series
assert type(df_ratings['timestamp']) is Series
assert type(df_tags['timestamp']) is Series

assert all(type(dt) is pd.Timestamp for dt in df_ratings['datetime'])
assert all(type(dt) is pd.Timestamp for dt in df_tags['datetime'])

partial_solution = pd.read_msgpack(b'\x87\xa3typ\xa6series\xa5klass\xa6Series\xa4name\xa9timestamp\xa5index\x86\xa3typ\xabrange_index\xa5klass\xaaRangeIndex\xa4name\xc0\xa5start\x00\xa4stop\x05\xa4step\x01\xa5dtype\xa5int64\xa4data\xc7(\x00\xafw\x849\x00\x00\x00\x00\xffq\x849\x00\x00\x00\x00\xd0u\x849\x00\x00\x00\x00\x07|\x849\x00\x00\x00\x00\x93x\x849\x00\x00\x00\x00\xa8compress\xc0')
pd.testing.assert_series_equal(df_ratings["timestamp"].head(), partial_solution)

partial_solution = pd.read_msgpack(b'\x87\xa3typ\xa6series\xa5klass\xa6Series\xa4name\xa9timestamp\xa5index\x86\xa3typ\xabrange_index\xa5klass\xaaRangeIndex\xa4name\xc0\xa5start\x00\xa4stop\x05\xa4step\x01\xa5dtype\xa5int64\xa4data\xc7(\x002\xdc+V\x00\x00\x00\x004\xdc+V\x00\x00\x00\x000\xdc+V\x00\x00\x00\x00\x07\xdd+V\x00\x00\x00\x00\x00\xdd+V\x00\x00\x00\x00\xa8compress\xc0')
pd.testing.assert_series_equal(df_tags["timestamp"].head(), partial_solution)


#### Vraag 2: Tags opschonen
Tags hebben op dit moment allemaal verschillend hoofdlettergebruik. Dit is niet handig bij het verwerken van de tags. Zet ze daarom om naar enkel kleine letters. Hiervoor kun je de functie `str.lower` gebruiken.

Ook bij deze opdracht kan je goed de `.map()` functie inzetten.

In [ ]:
# TODO


In [ ]:
assert type(df_tags['tag']) is Series
assert all(isinstance(tag, str) for tag in df_tags['tag'])
assert all(tag.islower() for tag in df_tags['tag'])

### Tags aan films toewijzen
Nu we de tags opgeschoond hebben, kunnen we ook de tags toevoegen aan `df_movies`; zo hoeven we niet iedere keer te zoeken. 

Dit hebben we hieronder voor je geprogrammeerd. Daarbij maken we gebruik van de `groupby`, `aggregate` en `join` functies van `pandas`. Kijk even goed naar hoe we dit doen. Je zult deze functies niet nodig hebben bij de volgende opdrachten. Wel zijn ze handig en als je later meer gebruik maakt van `pandas` zul je ongetwijfeld deze functies zien terugkomen en gaan gebruiken. 

In [ ]:
# Groepeer alle data uit `df_tags` met dezelfde movieId
groups = df_tags.groupby('movieId')

# Pak daarvan enkel de `tag`
tags = groups['tag']

# Verzamel vervolgens alle informatie in een set 
tags = tags.aggregate(set)

display(tags.head())

In [ ]:
# Als de `tag` kolom eerder al is toegekend, haal deze weg
if 'tag' in df_movies:
    del df_movies['tag']

# Voeg samen (`join`) het dataframe `df_movies` en de Series `tags` op de kolom `movieId`
df_movies = df_movies.join(tags, on='movieId')

display(df_movies.head())

#### Vraag 3: Genres verwerken
De huidige vorm van de genres (gescheiden door `|` in één string) is erg onhandig. We gaan de genres net als de timestamps omzetten naar iets wat we makkelijker kunnen gebruiken. Aan jou de taak om alle genres van een film in een `set` te zetten in de kolom `genres` (net zoals bij de kolom `tag`).

Je hoeft hier geen gebruik te maken van `groupby`, `aggregate` of `join`.

In [ ]:
# TODO


In [ ]:
assert type(df_movies["genres"]) is Series
assert all(isinstance(genre, set) for genre in df_movies["genres"])

partial_solution = pd.read_msgpack(b'\x87\xa3typ\xa6series\xa5klass\xa6Series\xa4name\xa6genres\xa5index\x86\xa3typ\xabrange_index\xa5klass\xaaRangeIndex\xa4name\xc0\xa5start\x00\xa4stop\x05\xa4step\x01\xa5dtype\xa6object\xa4data\x95\x95\xa9Adventure\xa9Animation\xa8Children\xa6Comedy\xa7Fantasy\x93\xa9Adventure\xa8Children\xa7Fantasy\x92\xa6Comedy\xa7Romance\x93\xa6Comedy\xa5Drama\xa7Romance\x91\xa6Comedy\xa8compress\xc0')

#Msgpack does not support sets, so we used a list. We now transform that list into a set.
partial_solution = partial_solution.map(set)

pd.testing.assert_series_equal(df_movies["genres"].head(), partial_solution)


### Jaren uit titels halen
Een interessant kenmerk van een film is het jaar waarin de film is uitgebracht. In onze dataset staat dit jaar altijd aan het einde van de titel tussen twee haken; `Toy Story (1995)`. Hiervoor kunnen we een reguliere expressie (regex) gebruiken om het jaar uit de titel te halen. Reguliere expressies bieden een manier om strings te analyseren en stukken er uit te filteren.

#### Regex
In essentie is het idee van een reguliere expressie simpel. Je schrijft een patroon en je kan dit patroon vervolgens vergelijken (pattern matchen) op strings. Hoe je zo'n patroon schrijft is even oefenen en dat gaat het best buiten een notebook. Ga hiervoor naar <https://regexone.com/> en werk door alle opdrachten. Geen zorgen, ze zijn niet lang en je kan altijd heen en weer springen.

Nu je een beetje hebt geoefend met reguliere expressies is het tijd om ze in te zetten op de titels van de dataset. Python kent net zoals eigenlijk elke programmeertaal een implementatie voor reguliere expressies. In Python vind je deze in de module `re` (drie keer raden waar dat kort voor is). Mocht je later gebruik maken van een andere programmeertaal zoals bijvoorbeeld JavaScript, dan zul je merken dat het idee rondom reguliere expressies consistent is, maar er altijd subtiele verschillen zijn. Wat je nu oefent kan je dus overdragen naar een andere taal / omgeving, maar het zal niet altijd met copy+paste meteen werken.

Een reguliere expressie in Python is simpelweg een string. Er is geen speciaal datatype. Wel kan het handig zijn om een zogenaamde raw string literal te gebruiken. In een raw string literal wordt elk teken letterlijk genomen. Bijvoorbeeld `r"\n"` is een raw string met daarin een backslash en de letter n, niet een newline zoals je bij `"\n"` wel zou krijgen. De functie waar je waarschijnlijk het meest aan hebt in deze opdracht is `re.search`. Kijk even naar het code voorbeelden hieronder:

In [ ]:
import re

regex = r"\w*"
match = re.search(regex, "foo 123 bar")

print(match)
print(match.span())
print(match.group(0))

In [ ]:
regex = r"bar (\w*)"
match = re.search(regex, "foo bar baz")

print(match)
print(match.span())
print(match.group(0))
print(match.group(1))

`re.search` geeft je een `re.Match` object terug. Een datastructuur voor een match. Daarin vind je de eerste match voor de meegeleverde regex. Wil je liever alle matches, dan kun je gebruik maken van `re.findall`. In een `re.Match` object zit een `.span()` methode die je aangeeft op welke indices er is gematched. Ook een `.group()` methode waarmee je alle groepen kan krijgen die zijn gematched. De eerste groep, groep 0, is de gehele match en vervolgens de subgroepen.

#### Vraag 4: Jaren ophalen

Nu is het aan jou om door middel van reguliere expressies het jaar uit de titel te halen en deze als integer in een aparte kolom `year` in `df_movies` te zetten.

Om uit te proberen waar jouw reguliere expressie matched kan je gebruik maken van deze website: <https://regex101.com/>. Daar kan je voorbeelden in typen en er reguliere expressies op los laten. Zo kan je goed nagaan wat er wordt gematched en wat niet.

Let op:
- Er zijn ook films waar het jaar niet het enige is wat tussen haken staat
- Er zijn een aantal films (11) waar er ook nog een spatie achteraan staat
- Er zijn een aantal films (12) die geen jaartal hebben, vul hier de waarde `None` voor in
- De film `Death Note: Desu nôto (2006-2007)` heeft twee jaartallen. Pak hierbij de laatste.

In [ ]:
# TODO


In [ ]:
assert type(df_movies["year"]) is Series

for year in df_movies["year"]:
    float(year)

assert all(1902 <= year <= 2018 or pd.isna(year) for year in df_movies["year"])

partial_solution = pd.read_msgpack(b'\x87\xa3typ\xa6series\xa5klass\xa6Series\xa4name\xa4year\xa5index\x86\xa3typ\xabrange_index\xa5klass\xaaRangeIndex\xa4name\xc0\xa5start\x00\xa4stop\x05\xa4step\x01\xa5dtype\xa7float64\xa4data\xc7(\x00\x00\x00\x00\x00\x00,\x9f@\x00\x00\x00\x00\x00,\x9f@\x00\x00\x00\x00\x00,\x9f@\x00\x00\x00\x00\x00,\x9f@\x00\x00\x00\x00\x00,\x9f@\xa8compress\xc0')
pd.testing.assert_series_equal(df_movies["year"].head(), partial_solution)

## De beste films

Wat zijn de best beoordeelde films?

Op de oppervlak een simpele vraag, maar er gaat veel meer achter schuil. Is een film uit 1980 even relevant als een film uit 2010? Kunnen we beter gemiddeldes vergelijken of kijken naar de mediaan? Is een film met één rating van vijf sterren en dus vijf sterren gemiddeld, beter dan een film met 1000 ratings en vier sterren gemiddeld? Is een film met overwegend recente ratings interressanter dan een film met oudere ratings? Hoe zijn de ratings per film verdeeld, gepolariseerd misschien?

Om een goed antwoord op de vraag te krijgen zul je in de data moeten duiken en inzicht moeten opbouwen. Dat ga je in dit deel van de opdracht doen.

#### Vraag 5: Gemiddelde en mediaan

Implementeer de functies `mean` en `median` hieronder. De functies accepteren een `DataFrame` een index om op te groeperen en een index om het gemiddelde/mediaan van uit te rekenen. De functies returnen een `Series` met als index de groepen en als datavelden de gemiddeldes/mediaan respectievelijk.

Zo berekent deze aanroep:

```Py
mean(df_ratings, "movieId", "rating")
```

De gemiddelde `rating` per `movieId` uit het `DataFrame` `df_ratings`.

In [ ]:
def mean(frame, group_index, avg_index):
    # TODO
    
    
def median(frame, group_index, avg_index):
    # TODO
    

In [ ]:
partial_solution = pd.read_msgpack(b'\x87\xa3typ\xa6series\xa5klass\xa6Series\xa4name\xa6rating\xa5index\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xa7movieId\xa5dtype\xa5int64\xa4data\xc7(\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa5dtype\xa7float64\xa4data\xc7(\x00\x0b\xe1u\xab\x10^\x0f@t\xd1E\x17]t\x0b@;\xb1\x13;\xb1\x13\n@\xb7m\xdb\xb6m\xdb\x02@%I\x92$I\x92\x08@\xa8compress\xc0')
pd.testing.assert_series_equal(mean(df_ratings, "movieId", "rating").head(), partial_solution)

partial_solution = pd.read_msgpack(b'\x87\xa3typ\xa6series\xa5klass\xa6Series\xa4name\xa6rating\xa5index\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xa7movieId\xa5dtype\xa5int64\xa4data\xc7(\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa5dtype\xa7float64\xa4data\xc7(\x00\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x0c@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x08@\xa8compress\xc0')
pd.testing.assert_series_equal(median(df_ratings, "movieId", "rating").head(), partial_solution)


Nu we deze functies hebben kunnen we makkelijk gemiddeldes en medianen uitrekenen per groep van data. Dit kunnen we vervolgens inzichtelijk maken met histogrammen. In essentie is dit erg simpel met `pandas`. Zo maakt:

```Py
Series([1,2,3,4,5]).hist()
```

een histogram voor de series 1,2,3,4,5. Gezien de functies `mean` en `median` beide een `Series` returnen kunnen we hiervoor ook histogrammen maken. Om ze met elkaar te vergelijken hebben we ze voor je naast elkaar gezet hieronder:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Maak een figuur aan met daarin 2 subplots, deel de y-as
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)

# Geef meer ruimte tussen de subplots
fig.subplots_adjust(right=2)

# Laat de y-as aan de rechterkant staan van de eerste subplot
ax1.yaxis.tick_right()

# Plot medians in eerste subplot
rating_medians = median(df_ratings, "movieId", "rating")
rating_medians.hist(ax=ax1, bins=np.linspace(0.5, 5, 10))
ax1.set_xlabel("ratings")
ax1.set_ylabel("occurances")
ax1.set_title("median ratings")

# Plot means in tweede subplot
rating_means = mean(df_ratings, "movieId", "rating")
rating_means.hist(ax=ax2, bins=np.linspace(0.5, 5, 10))
ax2.set_xlabel("ratings")
ax2.set_ylabel("occurances")
ax2.set_title("mean ratings")

# Laat figuur zien
plt.show()

Er is iets vreemds aan de hand hier! Zo zijn er aardig wat films met een 5 of 0.5 als mediaan of als gemiddelde. De kans daarop is natuurlijk vrij klein. Dit betekent waarschijnlijk dat er veel films tussen zitten met maar weinig reviews. Kijk maar:

In [ ]:
# Groepeer per movieId
groups = df_ratings.groupby("movieId")

# Pak enkel de ratings
ratings = groups["rating"]

# Bereken hoeveel ratings er zijn
ratings = ratings.apply(len)

# Maak een figuur aan met daarin 3 subplots
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3)

# Geef meer ruimte tussen de subplots
fig.subplots_adjust(right=2)

# Plot ratings per film op volgorde van aantal ratings
ax1.plot(list(ratings.sort_values(ascending=False)))
ax1.grid(True)
ax1.set_xlabel("movies")
ax1.set_ylabel("number of ratings")
ax1.set_title("ratings per movie")

# Plot ratings per film op volgorde van aantal ratings met een logaritmische schaal
ax2.plot(list(ratings.sort_values(ascending=False)))
ax2.grid(True)
ax2.set_yscale("log")
ax2.set_xlabel("movies")
ax2.set_ylabel("number of ratings")
ax2.set_title("ratings per movie")

# Plot histogram van ratings
ratings.hist(ax=ax3)
ax3.grid(True)
ax3.set_xlabel("number of ratings")
ax3.set_ylabel("occurances")
ax3.set_title("occurance of number of ratings")

# Laat figuur zien
fig.show()

We hebben hier te maken met een "long tail". De ratings per movie volgen een "power law distribution". Dit kan je goed terug zien in de logaritmische plot.  Er zijn dus heel veel films met weinig ratings. Dit zijn films waarover we op basis van ratings maar weinig weten. Deze kunnen we daarom beter niet meenemen in de zoektocht naar de best beoordeelde films.

#### Vraag 6: Ondergrens

Aan jou de taak om een harde grens te implementeren. Filter alle films met minder dan 10 reviews uit `df_ratings` en sla het nieuwe `DataFrame` op in `df_ratings_filtered`. Hercreër vervolgens de median en mean histogrammeren van eerder met de gefilterde data.

In [ ]:
# TODO


In [ ]:
assert type(df_ratings_filtered) is DataFrame

partial_solution = pd.read_msgpack(b"\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xa5Index\xa4name\xc0\xa5dtype\xa6object\xa4data\x95\xa6userId\xa7movieId\xa6rating\xa9timestamp\xa8datetime\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7(\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x93\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x01\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xd7\x00\x02\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7(\x00\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x14@\xa5shape\x92\x01\x05\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x03\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7x\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00/\x00\x00\x00\x00\x00\x00\x002\x00\x00\x00\x00\x00\x00\x00\xafw\x849\x00\x00\x00\x00\xffq\x849\x00\x00\x00\x00\xd0u\x849\x00\x00\x00\x00\x07|\x849\x00\x00\x00\x00\x93x\x849\x00\x00\x00\x00\xa5shape\x92\x03\x05\xa5dtype\xa5int64\xa5klass\xa8IntBlock\xa8compress\xc0\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x01\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xd7\x00\x04\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7(\x00\x00V'<4Ud\r\x00v\xca;\xe1Sd\r\x00`\x87\xb5\xc4Td\r\x00\xc6\x84$7Vd\r\x00>\x03RiUd\r\xa5shape\x92\x01\x05\xa5dtype\xaedatetime64[ns]\xa5klass\xadDatetimeBlock\xa8compress\xc0")
pd.testing.assert_frame_equal(df_ratings_filtered.head(), partial_solution)


#### Vraag 7: Top 5

Gegeven `df_ratings_filtered`, wat is de top 5 best gereviewde films? Kijk daarbij naar de gemiddelde rating.

Sla de top 5 als `Series` op in `top_5_ratings` met als index `movieId`

In [ ]:
# TODO


In [ ]:
assert type(top_5_ratings) is Series

solution = pd.read_msgpack(b'\x87\xa3typ\xa6series\xa5klass\xa6Series\xa4name\xa6rating\xa5index\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xa7movieId\xa5dtype\xa5int64\xa4data\xc7(\x00\x11\x04\x00\x00\x00\x00\x00\x00{\r\x00\x00\x00\x00\x00\x00\x9a\x04\x00\x00\x00\x00\x00\x00P\x04\x00\x00\x00\x00\x00\x008\t\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa5dtype\xa7float64\xa4data\xc7(\x00]t\xd1E\x17]\x12@/\xba\xe8\xa2\x8b.\x12@\xab\xaa\xaa\xaa\xaa*\x12@fffff\xe6\x11@UUUUU\xd5\x11@\xa8compress\xc0')
pd.testing.assert_series_equal(top_5_ratings, solution, check_names=False)

### Een betere rating

Gebruikers reviewen niet allemaal op dezelfde manier. Zo geeft de één al snel vijf sterren voor een film en vindt de ander drie sterren ook wel positief. Ook zegt een vijf sterren review van een user die altijd vijf sterren review niet zo veel over de film. Dit kunnen we meenemen bij de zoektocht naar de beste film. Laten we allereerst een blik werpen op de data:

In [ ]:
user_ratings = df_ratings.groupby("userId").apply(len).sort_values(ascending=False)

# Maak een figuur aan met daarin 2 subplots, deel de y-as
fig, (ax1, ax2) = plt.subplots(ncols=2)

# Geef meer ruimte tussen de subplots
fig.subplots_adjust(right=2)

# Plot users vs n_ratings
ax1.plot(list(user_ratings))
ax1.set_xlabel("users")
ax1.set_ylabel("number of ratings")

# Plot n_users per gemiddelde rating
ax2.hist(list(mean(df_ratings, "userId", "rating").sort_values(ascending=False)), bins=np.linspace(0.5, 5, 10))
ax2.set_xlabel("average rating")
ax2.set_ylabel("number of users")

# Laat figuur zien
plt.show()

Dit komt bekend voor! Ook hier is in de linkergrafiek een power-law distributie te zien. Kijken we naar de gemiddelde rating, dan zien we dat de bulk van de gebruikers gemiddeld een rating geeft van ongeveer 3.5 t/m 4.2, maar lang niet iedereen!

#### Vraag 8: Zeer kritisch

Welke gebruiker geeft er gemiddeld lager dan een 2 als rating? Voor welke films? Sla alle door die gebruiker gereviewde films op in een `list` genaamd `titles`.

In [ ]:
# TODO


In [ ]:
assert isinstance(titles, list)

solution = pd.read_msgpack(b"\x87\xa3typ\xa6series\xa5klass\xa6Series\xa4name\xa5title\xa5index\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\xa0\x00\xdd\n\x01\x00\x00\x00\x00\x00\xde\n\x01\x00\x00\x00\x00\x00\xdf\n\x01\x00\x00\x00\x00\x00\xe0\n\x01\x00\x00\x00\x00\x00\xe1\n\x01\x00\x00\x00\x00\x00\xe2\n\x01\x00\x00\x00\x00\x00\xe3\n\x01\x00\x00\x00\x00\x00\xe4\n\x01\x00\x00\x00\x00\x00\xe5\n\x01\x00\x00\x00\x00\x00\xe6\n\x01\x00\x00\x00\x00\x00\xe7\n\x01\x00\x00\x00\x00\x00\xe8\n\x01\x00\x00\x00\x00\x00\xe9\n\x01\x00\x00\x00\x00\x00\xea\n\x01\x00\x00\x00\x00\x00\xeb\n\x01\x00\x00\x00\x00\x00\xec\n\x01\x00\x00\x00\x00\x00\xed\n\x01\x00\x00\x00\x00\x00\xee\n\x01\x00\x00\x00\x00\x00\xef\n\x01\x00\x00\x00\x00\x00\xf0\n\x01\x00\x00\x00\x00\x00\xa8compress\xc0\xa5dtype\xa6object\xa4data\xdc\x00\x14\xb7Jungle Book, The (1994)\xd9BEnglishman Who Went Up a Hill But Came Down a Mountain, The (1995)\xabRudy (1993)\xb2Heavy Metal (1981)\xb6Aristocats, The (1970)\xbfSex, Lies, and Videotape (1989)\xb7Right Stuff, The (1983)\xadPatton (1970)\xd9&I Know What You Did Last Summer (1997)\xb9Dangerous Liaisons (1988)\xb5Pretty in Pink (1986)\xb6Double Jeopardy (1999)\xb5Boys Don't Cry (1999)\xb0Backdraft (1991)\xb8American Graffiti (1973)\xaaJFK (1991)\xb0Frequency (2000)\xb9Me, Myself & Irene (2000)\xb0Cell, The (2000)\xaeTootsie (1982)\xa8compress\xc0")
solution = set(solution)
for title in titles:
    assert title in solution, f"did not expect {title}"
for title in solution:
    assert title in titles, f"missing {title}"

Gebruikers hebben een bias, ze reviewen niet op dezelfde manier en daardoor is een absolute rating minder interessant. Om een beter systeem te maken kunnen we kijken naar de gemiddelde rating van een gebruiker en hoe de gegeven rating afwijkt van dit gemiddelde. Zo weet je of een gebruiker relatief een film beter of slechter vind. In andere woorden "mean-centeren" we de rating. We nemen het gemiddelde, de mean, als middenpunt en herschrijven de rating naar de afwijking van dit middenpunt.

Zodra je de afwijking van het middenpunt weet, kan je dit inzetten om films te vergelijken. Zo kun je het gemiddelde nemen van de afwijking van elke rating van een film en dit gebruiken als nieuwe beoordeling van een film.

#### Vraag 9: Een betere top 5

Aan jou nu de taak om een nieuwe top 5 op te stellen. Dit keer niet op basis van de gemiddelde rating, maar op basis van de gemiddelde mean-centered rating. Gebruik ook hier weer `df_ratings_filtered` als dataset. Sla de top 5 op als `Series` in een variabele genaamd `top_5_mean_centered` met als index `movieId`.

Let op, deze opdracht is iets groter. Bedenk van te voren goed welke stappen je moet doorlopen.

In [ ]:
# TODO


In [ ]:
assert type(top_5_mean_centered) is Series

solution = pd.read_msgpack(b"\x87\xa3typ\xa6series\xa5klass\xa6Series\xa4name\xabdeltaRating\xa5index\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xa7movieId\xa5dtype\xa5int64\xa4data\xc7(\x00{\r\x00\x00\x00\x00\x00\x00\x9a\x04\x00\x00\x00\x00\x00\x00\xb7\x03\x00\x00\x00\x00\x00\x00\xe0\x04\x00\x00\x00\x00\x00\x00P\x04\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa5dtype\xa7float64\xa4data\xc7(\x00\xaf\x82'\xe73|\xee?\xfd\x9b\xc0\xd0#\x13\xeb?\xd5\xfb\xaf\x03X\xfe\xe8?\x96(j\x96\xe1?\xe8?R\xed\x1cL\xfe5\xe8?\xa8compress\xc0")
pd.testing.assert_series_equal(top_5_mean_centered, solution, check_names=False)